<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Rentabilidad_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [418]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

In [419]:
archivo = 'Resultados ML.xlsx'

# Archivos necesarios

In [420]:
df = pd.read_excel('MercadoLibre.xlsx', sheet_name=0, header=5)
df.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,# de venta,Fecha de venta,Estado,Descripción del estado,Paquete de varios productos,Pertenece a un kit,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos (MXN),Ingresos por envío (MXN),...,Revisado por Mercado Libre,Fecha de revisión,Dinero a favor,Resultado,Destino,Motivo del resultado,Unidades.2,Reclamo abierto,Reclamo cerrado,Con mediación
0,2000008525236973,16 de julio de 2025 07:57 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
1,2000012317175376,16 de julio de 2025 06:05 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,No,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
2,2000008523351917,16 de julio de 2025 02:51 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
3,2000008523277155,16 de julio de 2025 01:46 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
4,2000012316262142,16 de julio de 2025 01:46 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,No,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No


Del renglonaje, extraemos las columnas de los costos de los productos y los SKUs

In [421]:
renglonaje = pd.read_excel('Renglonaje.xlsx', sheet_name=0)
renglonaje.rename(columns={'Total que paga Edy a Truper contando reserva en garantía CON IVA': 'Costo a Truper', 'clave':'SKU'}, inplace=True)
precios = renglonaje[['SKU', 'Costo a Truper']]
precios = precios.loc[precios['Costo a Truper'] != 'No disponible']
precios['Costo a Truper'] = precios['Costo a Truper'].astype(float)
precios.head()

,SKU,Costo a Truper
0,PET-15X,290.92460
1,PET-15C,407.29444
2,REP-CUT-5X,13.96797
3,REP-CUT-6X,24.82530
4,REP-CUT-650,65.94158


Creamos las columnas en las posiciones deseadas

In [422]:
df.insert(13, 'Costo a Truper', '')
df.insert(14, 'Costo Skydropx', '')
df.insert(15, '¿Zona extendida?', '')
df.insert(16, 'Rentabilidad', '')
df.insert(17, '¿Rentable?', '')

## Extracción del costo a truper del renglonaje

In [423]:
df['Costo a Truper'] = df.merge(precios, on='SKU', how='left')['Costo a Truper_y']
df.head()

,# de venta,Fecha de venta,Estado,Descripción del estado,Paquete de varios productos,Pertenece a un kit,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos (MXN),Ingresos por envío (MXN),...,Revisado por Mercado Libre,Fecha de revisión,Dinero a favor,Resultado,Destino,Motivo del resultado,Unidades.2,Reclamo abierto,Reclamo cerrado,Con mediación
0,2000008525236973,16 de julio de 2025 07:57 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
1,2000012317175376,16 de julio de 2025 06:05 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,No,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
2,2000008523351917,16 de julio de 2025 02:51 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
3,2000008523277155,16 de julio de 2025 01:46 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,Sí,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No
4,2000012316262142,16 de julio de 2025 01:46 hs.,Etiqueta lista para imprimir,Tienes que entregar el paquete hoy antes de la...,No,No,1,1550,-302.25,NaN,...,,,,,,,NaN,No,NaN,No


## Extracción del costo de Skydropx

In [424]:
df['Costo Skydropx'] = df['Ingresos por envío (MXN)']
df['Costo Skydropx'].fillna(0, inplace=True)

/tmp/ipython-input-424-447394439.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Costo Skydropx'].fillna(0, inplace=True)
/tmp/ipython-input-424-447394439.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

## Extracción de zonas extendidas

In [ ]:
df['¿Zona extendida?'].replace('', 0, inplace=True)

## Cálculo de Rentabilidad

In [426]:
df['Rentabilidad'] = df['Total (MXN)'] - (df['Costo a Truper'] * df['Unidades']) - (df['Costo Skydropx'] + df['¿Zona extendida?'])

# Definimos rentabilidad
Si el producto nos da una rentabilidad mayor a 0, se evalúa como Sí rentable; si no, se evalúa como No rentable.

In [427]:
df.loc[df['Rentabilidad'] > 0.0, '¿Rentable?'] = 'Sí'
df.loc[df['Rentabilidad'] <= 0.0, '¿Rentable?'] = 'No'

# Creacion del Excel

In [428]:
df.to_excel(archivo, index=False)

# Formato del excel

In [429]:
wb = load_workbook('Resultados ML.xlsx')
ws = wb.active

## Colores de fondo
- fill_y = Amarillo
- fill_o = Naranja
- fill_lb = Azul claro
- fill_lgy = Gris claro
- fill_lgr = Verde claro
- fill_r = Rojo
- fill_b = Negro

In [430]:
fill_y = PatternFill(fill_type="solid", start_color="f1c232", end_color="f1c232")
fill_o = PatternFill(fill_type="solid", start_color="f6b26b", end_color="f6b26b")
fill_lb = PatternFill(fill_type="solid", start_color="6d9eeb", end_color="6d9eeb")
fill_lgy = PatternFill(fill_type="solid", start_color="b7b7b7", end_color="b7b7b7")
fill_lgr = PatternFill(fill_type="solid", start_color="93c47d", end_color="93c47d")
fill_r = PatternFill(fill_type="solid", start_color="e06666", end_color="e06666")
fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")

## Fuentes

In [431]:
font_t = Font(name='Arial', size=12, bold=True)
font_tw = Font(name='Arial', size=12, bold=True, color='ffffff')
font_b = Font(name='Arial', size=12, bold=False)

## Ancho de columnas

In [432]:
for col in ['A', 'B', 'C', 'D', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R']:
    ws.column_dimensions[col].width = 18

## Color del header

In [433]:
for col in range(1, 66):
  ws.cell(row=1, column=col).fill = fill_y
  ws.cell(row=1, column=col).font = font_t
  ws.cell(row=1, column=col).alignment = Alignment(horizontal='right')
for row in range(2, 1500):
  for col in range(1, 66):
    ws.cell(row=row, column=col).font = font_b
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='right')

## Formato numérico

In [434]:
num_form = '$ #,##0.00'
for row in range(2, 1500):
  for col in range(8, 18):
    ws.cell(row=row, column=col).number_format = num_form

## Color de columnas
Se colorean las columnas de Costo a Truper, Skydropx, Zona extendida y Rentabilidad (en ese orden).

In [435]:
for row in range(1, 1500):
  if ws.cell(row=row, column=13).value == None:
    break
  ws.cell(row=row, column=14).fill = fill_o
  ws.cell(row=row, column=15).fill = fill_lb
  ws.cell(row=row, column=16).fill = fill_lgy
  ws.cell(row=row, column=17).fill = fill_b
  ws.cell(row=row, column=17).font = font_tw

### Color de columna Rentable
Se colorea la columna de Rentable. Si el valor de la columna es "Sí", se colorea de verde; si no, se colorea de rojo.

In [436]:
ws.cell(row=1, column=18).fill = fill_lgy
for row in range(2, 1500):
  if ws.cell(row=row, column=18).value == None:
    break
  if ws.cell(row=row, column=18).value == 'Sí':
    ws.cell(row=row, column=18).fill = fill_lgr
  else:
    ws.cell(row=row, column=18).fill = fill_r

# Fin del Excel

In [437]:
wb.save(archivo)